# Preparation
The pre-requisites for the command server to work properly is that there must be a motion_server.py running (otherwise it has no way of connecting to it - thus this piece of code starts a command server for you, if you haven't )

In [ ]:
from motion_client import MotionClient()
import os,subprocess

try:
    a = MotionClient()

# Imports
Import both Jarvis and command_server (you won't need to import command server usually, as it should be always be running on TRINA_GPU, but if you're doing a local test, it has to be imported and instantiated)

In [24]:
# from command_server import CommandServer
from jarvis import Jarvis
import time

# Instantiate Jarvis and command server

Same thing here, on real trina, you only need to import jarvis, as the command_server would always be running

In [30]:
# command_server = CommandServer()
# jarvis = Jarvis('C2')
# j2 = Jarvis('C1')
# j3 = Jarvis('PointClickNav')
test = Jarvis('UI')
# test2 = Jarvis('test2')


In [31]:
print(test.server['ACTIVITY_STATUS'].read())


{u'PointClickNav': u'idle', u'C2': u'idle', u'C1': u'idle', u'UI': u'idle', u'DirectTeleOperation': u'idle'}


In [20]:
test.sensedRobotq()

[-3.2235968570015534,
 -1.6376341400571033,
 0,
 -2.8903999999999086,
 0,
 0,
 0,
 -0.2028,
 -2.1063,
 -1.61,
 3.7165,
 -0.9622,
 0.0974,
 0,
 0,
 0.2028,
 -1.035292653589793,
 1.61,
 -0.5749073464102068,
 0.9622,
 -0.0974,
 0]

In [18]:

# # for i in a.keys():
# #     k = jarvis.server['ACTIVITY_STATUS'][i]
print(test.server['ROBOT_COMMAND'].read())
# test.changeActivityStatus([str('C2')])
print(test.server['ROBOT_COMMAND'].read())

# print(jarvis.server['ROBOT_COMMAND'].read())
# time.sleep(0.01)



{u'PointClickNav': [], u'C2': [], u'C1': [], u'UI': []}
{u'PointClickNav': [], u'C2': [], u'C1': [], u'UI': []}


In [ ]:
for i in range(10):
#     test.setBaseVelocity([1,1])
#     jarvis.setBaseVelocity([0,0])
    j2.setBaseVelocity([0,0])
#     j3.setBaseVelocity([0,1])
    print(jarvis.server['ROBOT_COMMAND'].read())


In [ ]:
j2.setBaseVelocity([0.01,0])

# New heading

In [ ]:
from multiprocessing import Manager

manager = Manager()
d = manager.set()
d['a'] = 0

In [ ]:
dir(manager)